In [2]:
import pandas as pd
import os
import datetime

### 讀取檔案 ¶

In [3]:
def load_data(path,sep):
    doc=pd.read_csv(path,encoding='utf-8',sep=sep)
    # 整理日期轉成Pandas datetime 物件
    doc['時間']= pd.to_datetime(doc['時間'])
    # 新增對應股價的時間欄位
    doc["預計影響日期"] = doc["時間"]
    return doc
df = load_data(r'data/lstm_practice_data_all_0304.csv','{')

## 交易時間：早上9點~下午1點半 (下午兩點半後發的新聞算入隔天)
### 更改成每半小時
下午2點半後的新聞 影響隔天早上9點30(開盤)
<br>09:00-09:29 -- 09:30
<br>09:30-09:59 -- 10:00
<br>10:00-10:29 -- 10:30
<br>10:30-10:59 -- 11:00
<br>11:00-11:29 -- 11:30
<br>11:30-11:59 -- 12:00
<br>12:00-12:29 -- 12:30
<br>12:30-12:59 -- 13:00
<br>13:00-13:29 -- 13:30
<br>13:30-13:59 -- 09:00
<br>14:00-14:29 -- 09:00

In [5]:
#處理分鐘至00 or 30
def change_minute(doc):
    
    for idx, row in doc.iterrows():
        if doc.loc[idx,"時間"].minute < 30:
            doc.loc[idx,"預計影響日期"] = doc.loc[idx,"時間"].replace(minute=30, second=0, microsecond=0)

        elif doc.loc[idx,"時間"].minute >= 30:
            doc.loc[idx,"預計影響日期"] = doc.loc[idx,"時間"].replace(minute=0, second=0, microsecond=0)
            doc.loc[idx,"預計影響日期"] += pd.Timedelta(seconds=3600)
    return doc
df = change_minute(df)

In [7]:
def adjust_after_close(doc):    
    #設定股票收盤時間
    close_time = 1330   #收盤時間
    close_time2 = 1430  #盤後交易
    #處理時間超過盤後交易(13:40~14:30)的部分
    for idx, row in doc.iterrows():
        if doc.loc[idx,"預計影響日期"].hour*100 + doc.loc[idx,"預計影響日期"].minute > close_time2:
            doc.loc[idx,"預計影響日期"] += pd.Timedelta(days=1)
            doc.loc[idx,"預計影響日期"] = doc.loc[idx,"預計影響日期"].replace(hour=9, minute=30, second=0, microsecond=0)
    
    #處理時間超過收盤時間的部分
    for idx, row in doc.iterrows():
        if doc.loc[idx,"預計影響日期"].hour*100 + doc.loc[idx,"預計影響日期"].minute > close_time:
            doc.loc[idx,"預計影響日期"] += pd.Timedelta(days=1)
            doc.loc[idx,"預計影響日期"] = doc.loc[idx,"預計影響日期"].replace(hour=9, minute=0, second=0, microsecond=0)
    
    return doc 

df = adjust_after_close(df)

In [13]:
def adjust_before_open(doc):
    #處理時間不到0900的部分
    for idx, row in df_all.iterrows():
        if doc.loc[idx,"預計影響日期"].hour*100 < 900:
            doc.loc[idx,"預計影響日期"] = doc.loc[idx,"預計影響日期"].replace(hour=9, minute=30, second=0, microsecond=0)
    
    return doc

df = adjust_before_open(df)

In [14]:
def skip_holiday(doc):
    
    #2020國定假日休市
    close_day = [datetime.date(2020,1,1), datetime.date(2020,1,21), datetime.date(2020,1,22), datetime.date(2020,1,23),\
                 datetime.date(2020,1,24), datetime.date(2020,1,25), datetime.date(2020,1,26), datetime.date(2020,1,27),\
                 datetime.date(2020,1,28), datetime.date(2020,1,29), datetime.date(2020,2,28), datetime.date(2020,4,2),\
                 datetime.date(2020,4,3), datetime.date(2020,5,1), datetime.date(2020,6,25), datetime.date(2020,6,26),\
                 datetime.date(2020,10,1), datetime.date(2020,10,2), datetime.date(2020,10,9)]
    #週末但不休市
    open_day = []
    
    #處理國定假日休市
    for ind, row in doc.iterrows():
        while doc.loc[ind,"預計影響日期"] in close_day:
            doc.loc[ind,"預計影響日期"] += pd.Timedelta(days=1)
            doc.loc[ind,"預計影響日期"] = doc.loc[ind,"預計影響日期"].replace(hour=9, minute=30, second=0, microsecond=0)
    
    #處理週末休市
    for ind, row in df_all.iterrows():
        if doc.loc[ind,"預計影響日期"].weekday() == 6:
            doc.loc[ind,"預計影響日期"] += pd.Timedelta(days=1)
            doc.loc[ind,"預計影響日期"] = doc.loc[ind,"預計影響日期"].replace(hour=9, minute=30, second=0, microsecond=0)

        elif doc.loc[ind,"預計影響日期"].weekday() == 5 and doc.loc[ind,"預計影響日期"] not in open_day:
            doc.loc[ind,"預計影響日期"] += pd.Timedelta(days=2)

            doc.loc[ind,"預計影響日期"] = doc.loc[ind,"預計影響日期"].replace(hour=9, minute=30, second=0, microsecond=0)

        else:
            pass
    
    return doc

df = skip_holiday(df)

In [15]:
#處理國定假日休市
for ind, row in df_all.iterrows():
    while df_all.loc[ind,"預計影響日期"] in close_day:
        df_all.loc[ind,"預計影響日期"] += pd.Timedelta(days=1)
        df_all.loc[ind,"預計影響日期"] = df_all.loc[ind,"預計影響日期"].replace(hour=9, minute=30, second=0, microsecond=0)

In [16]:
#處理週末休市
for ind, row in df_all.iterrows():
    if df_all.loc[ind,"預計影響日期"].weekday() == 6:
        df_all.loc[ind,"預計影響日期"] += pd.Timedelta(days=1)
        df_all.loc[ind,"預計影響日期"] = df_all.loc[ind,"預計影響日期"].replace(hour=9, minute=30, second=0, microsecond=0)
    
        #print(df_all.loc[ind,"預計影響日期"])
    elif df_all.loc[ind,"預計影響日期"].weekday() == 5 and df_all.loc[ind,"預計影響日期"] not in open_day:
        df_all.loc[ind,"預計影響日期"] += pd.Timedelta(days=2)
        #print(df_all.loc[ind,"預計影響日期"])
        df_all.loc[ind,"預計影響日期"] = df_all.loc[ind,"預計影響日期"].replace(hour=9, minute=30, second=0, microsecond=0)
    
    else:
        pass

## 存檔

In [32]:
def save_data(path,sep):
    df.to_csv(path,sep=sep,index=False)
    return f"saved to {path}"
save_data('changed_time.csv',sep='{')